In [145]:
import requests
import pandas as pd


## Custom functions for API call

In [158]:
def get_api_resp(END_POINT, params=None):
    URI = BASE_URI + END_POINT
    if params:
        return requests.get(URI, params=params)
    else:
        return requests.get(URI)
    
    

def get_phenotype_nbrs(gene_name, params=None):
    END_POINT = '{}/{}/{}/{}'.format(NBR_END_POINT, NODE_TYPE, ATTRIBUTE, gene_name)
    result = get_api_resp(END_POINT, params=params)
    neighbors = result.json()
    disease_nbr_names = list(filter(None, list(map(lambda x:x['data']['properties']['name'] if x['data']['neo4j_type'] == 'Disease' else None, neighbors))))
    symptom_nbr_names = list(filter(None, list(map(lambda x:x['data']['properties']['name'] if x['data']['neo4j_type'] == 'Symptom' else None, neighbors))))
    phenotype_names = disease_nbr_names +  symptom_nbr_names

    disease_nbr_ids = list(filter(None, list(map(lambda x:x['data']['id'] if x['data']['neo4j_type'] == 'Disease' else None, neighbors))))
    symptom_nbr_ids = list(filter(None, list(map(lambda x:x['data']['id'] if x['data']['neo4j_type'] == 'Symptom' else None, neighbors))))
    phenotype_ids = disease_nbr_ids + symptom_nbr_ids

    phenotype_df = pd.DataFrame(zip(phenotype_ids, phenotype_names), columns=['phenotype_id', 'phenotype_name'])

    nbr_edge_list = []
    for item in neighbors:
        try:
            if (item['data']['source'] in phenotype_ids): 
                nbr_edge_list.append((item['data']['source'], item['data']['properties']))
            elif (item['data']['target'] in phenotype_ids):
                nbr_edge_list.append((item['data']['target'], item['data']['properties']))
        except:
            continue

    nbr_edge_df = pd.DataFrame(nbr_edge_list, columns=['phenotype_id', 'properties'])
    phenotype_nbr_df = pd.merge(phenotype_df, nbr_edge_df, on='phenotype_id').drop('phenotype_id', axis=1)
    return list(phenotype_nbr_df.to_records(index=False))


## API parameters

In [159]:
BASE_URI = 'https://spoke.rbvi.ucsf.edu'
NBR_END_POINT = '/api/v1/neighborhood/'


API_PARAMS = {
    'cutoff_DaG_diseases_sources': ['knowledge', 'experiments', 'textmining'],
    'cutoff_DaG_textmining': 3,
}

NODE_TYPE = 'Gene'
ATTRIBUTE = 'name'



## Input Gene name to get the corresponding associated phenotype


In [163]:
%%time

gene_name = 'BRCA2'

phenotype_nbrs = get_phenotype_nbrs(gene_name, params=API_PARAMS)



CPU times: user 53.1 ms, sys: 7.08 ms, total: 60.2 ms
Wall time: 1.01 s


## Printing the extracted neighbors as Markdown

In [164]:
markdown_output = "### Phenotypes associated with the gene {} in SPOKE:\n\n".format(gene_name)
for idx, (phenotype, data) in enumerate(phenotype_nbrs, start=1):
    markdown_output += f"{idx}. **{phenotype}**\n"
    for key, value in data.items():
        if isinstance(value, list):
            value = ', '.join(map(str, value))
            markdown_output += f"   - {key.capitalize()}: {value}\n"
        else:
            markdown_output += f"   - {key.capitalize()}: {value}\n"
    markdown_output += "\n"

from IPython.display import Markdown
display(Markdown(markdown_output))


### Phenotypes associated with the gene BRCA2 in SPOKE:

1. **nephroblastoma**
   - Sources: OMIM
   - O_inheritance: AD,SOMATIC
   - O_modifiers: 

2. **intestinal cancer**
   - Diseases_identifiers: https://diseases.jensenlab.org/Entity?documents=10&type1=9606&id1=ENSP00000369497&type2=-26&id2=DOID:10155
   - Diseases_scores: 6.049
   - Sources: DISEASES
   - Diseases_sources: textmining
   - Diseases_confidences: 3.024

3. **large intestine cancer**
   - Diseases_identifiers: https://diseases.jensenlab.org/Entity?documents=10&type1=9606&id1=ENSP00000369497&type2=-26&id2=DOID:5672
   - Diseases_scores: 6.035
   - Sources: DISEASES
   - Diseases_sources: textmining
   - Diseases_confidences: 3.018

4. **attention deficit hyperactivity disorder**
   - Source: HPO

5. **embryonal tumor with multilayered rosettes, C19MC-altered**
   - Source: HPO

6. **melancholic depression**
   - Source: HPO

7. **cerebral primitive neuroectodermal tumor**
   - Source: HPO

8. **uterine cancer**
   - Source: HPO

9. **dysgraphia**
   - Source: HPO

10. **thyroid cancer**
   - Source: HPO

11. **rectal benign neoplasm**
   - Source: HPO

12. **thyroid gland disease**
   - Source: HPO

13. **pituitary cancer**
   - Source: HPO

14. **skin cancer**
   - Source: HPO

15. **neurotic disorder**
   - Source: HPO

16. **pituitary carcinoma**
   - Source: HPO

17. **Ewing sarcoma**
   - Source: HPO

18. **neuroblastoma**
   - Source: HPO

19. **skin benign neoplasm**
   - Source: HPO

20. **intracranial hypertension**
   - Source: HPO

21. **medulloepithelioma**
   - Source: HPO

22. **thyroid gland carcinoma**
   - Source: HPO

23. **agnosia**
   - Source: HPO

24. **pituitary adenoma**
   - Source: HPO

25. **agraphia**
   - Source: HPO

26. **rectum cancer**
   - Source: HPO

27. **lingual-facial-buccal dyskinesia**
   - Source: HPO

28. **visual epilepsy**
   - Source: HPO

29. **speech disorder**
   - Source: HPO

30. **hepatocellular carcinoma**
   - Source: HPO

31. **stomach cancer**
   - Source: HPO

32. **thyroid adenoma**
   - Source: HPO

33. **melancholia**
   - Source: HPO

34. **epilepsy**
   - Source: HPO

35. **migraine**
   - Source: HPO

36. **lung squamous cell carcinoma**
   - Sources: GWAS
   - Gwas_pvalue: 1e-15

37. **estrogen-receptor negative breast cancer**
   - Sources: GWAS
   - Gwas_pvalue: 2e-13

38. **lung carcinoma**
   - Sources: GWAS
   - Gwas_pvalue: 2e-19

39. **squamous cell carcinoma**
   - Sources: GWAS
   - Gwas_pvalue: 5e-20

40. **organ system cancer**
   - Diseases_identifiers: https://diseases.jensenlab.org/Entity?documents=10&type1=9606&id1=ENSP00000369497&type2=-26&id2=DOID:0050686, MedlinePlus, MedlinePlus, MedlinePlus, MedlinePlus
   - Diseases_scores: 7.809, CURATED, CURATED, CURATED, CURATED
   - Sources: DISEASES
   - Diseases_sources: textmining, knowledge, knowledge, knowledge, knowledge
   - Diseases_confidences: 3.905, 5.0, 5.0, 5.0, 5.0

41. **hepatobiliary system cancer**
   - Diseases_identifiers: MedlinePlus
   - Diseases_scores: CURATED
   - Sources: DISEASES
   - Diseases_sources: knowledge
   - Diseases_confidences: 5.0

42. **breast disease**
   - Diseases_identifiers: https://diseases.jensenlab.org/Entity?documents=10&type1=9606&id1=ENSP00000369497&type2=-26&id2=DOID:3463, MedlinePlus
   - Diseases_scores: 7.913, CURATED
   - Sources: DISEASES
   - Diseases_sources: textmining, knowledge
   - Diseases_confidences: 3.956, 5.0

43. **malignant ovarian surface epithelial-stromal neoplasm**
   - Diseases_identifiers: https://diseases.jensenlab.org/Entity?documents=10&type1=9606&id1=ENSP00000369497&type2=-26&id2=DOID:2151
   - Diseases_scores: 6.195
   - Sources: DISEASES
   - Diseases_sources: textmining
   - Diseases_confidences: 3.098

44. **carcinoma**
   - Diseases_identifiers: https://diseases.jensenlab.org/Entity?documents=10&type1=9606&id1=ENSP00000369497&type2=-26&id2=DOID:305, MedlinePlus
   - Diseases_scores: 6.797, CURATED
   - Sources: DISEASES
   - Diseases_sources: textmining, knowledge
   - Diseases_confidences: 3.398, 5.0

45. **gastrointestinal system cancer**
   - Diseases_identifiers: https://diseases.jensenlab.org/Entity?documents=10&type1=9606&id1=ENSP00000369497&type2=-26&id2=DOID:3119, MedlinePlus
   - Diseases_scores: 6.204, CURATED
   - Sources: DISEASES
   - Diseases_sources: textmining, knowledge
   - Diseases_confidences: 3.102, 5.0

46. **cholangiocarcinoma**
   - Diseases_identifiers: MedlinePlus
   - Diseases_scores: CURATED
   - Sources: DISEASES
   - Diseases_sources: knowledge
   - Diseases_confidences: 5.0

47. **ovarian carcinoma**
   - Diseases_identifiers: https://diseases.jensenlab.org/Entity?documents=10&type1=9606&id1=ENSP00000369497&type2=-26&id2=DOID:4001
   - Diseases_scores: 6.182
   - Sources: DISEASES
   - Diseases_sources: textmining
   - Diseases_confidences: 3.091

48. **ovary epithelial cancer**
   - Diseases_identifiers: https://diseases.jensenlab.org/Entity?documents=10&type1=9606&id1=ENSP00000369497&type2=-26&id2=DOID:2152
   - Diseases_scores: 6.559
   - Sources: DISEASES
   - Diseases_sources: textmining
   - Diseases_confidences: 3.279

49. **male reproductive organ cancer**
   - Diseases_identifiers: https://diseases.jensenlab.org/Entity?documents=10&type1=9606&id1=ENSP00000369497&type2=-26&id2=DOID:3856, MedlinePlus
   - Diseases_scores: 6.528, CURATED
   - Sources: DISEASES
   - Diseases_sources: textmining, knowledge
   - Diseases_confidences: 3.264, 5.0

50. **syndrome**
   - Diseases_identifiers: https://diseases.jensenlab.org/Entity?documents=10&type1=9606&id1=ENSP00000369497&type2=-26&id2=DOID:225
   - Diseases_scores: 6.794
   - Sources: DISEASES
   - Diseases_sources: textmining
   - Diseases_confidences: 3.397

51. **physical disorder**
   - Diseases_identifiers: https://diseases.jensenlab.org/Entity?documents=10&type1=9606&id1=ENSP00000369497&type2=-26&id2=DOID:0080015, MedlinePlus, UniProtKB-KW
   - Diseases_scores: 6.239, CURATED, CURATED
   - Sources: DISEASES
   - Diseases_sources: textmining, knowledge, knowledge
   - Diseases_confidences: 3.12, 5.0, 4.0

52. **congenital hypoplastic anemia**
   - Diseases_identifiers: https://diseases.jensenlab.org/Entity?documents=10&type1=9606&id1=ENSP00000369497&type2=-26&id2=DOID:1342, MedlinePlus, UniProtKB-KW
   - Diseases_scores: 7.057, CURATED, CURATED
   - Sources: DISEASES
   - Diseases_sources: textmining, knowledge, knowledge
   - Diseases_confidences: 3.528, 5.0, 4.0

53. **bile duct disease**
   - Diseases_identifiers: MedlinePlus
   - Diseases_scores: CURATED
   - Sources: DISEASES
   - Diseases_sources: knowledge
   - Diseases_confidences: 5.0

54. **prostate disease**
   - Diseases_identifiers: https://diseases.jensenlab.org/Entity?documents=10&type1=9606&id1=ENSP00000369497&type2=-26&id2=DOID:47, MedlinePlus
   - Diseases_scores: 6.513, CURATED
   - Sources: DISEASES
   - Diseases_sources: textmining, knowledge
   - Diseases_confidences: 3.257, 5.0

55. **cell type cancer**
   - Diseases_identifiers: https://diseases.jensenlab.org/Entity?documents=10&type1=9606&id1=ENSP00000369497&type2=-26&id2=DOID:0050687, MedlinePlus
   - Diseases_scores: 6.881, CURATED
   - Sources: DISEASES
   - Diseases_sources: textmining, knowledge
   - Diseases_confidences: 3.44, 5.0

56. **reproductive system disease**
   - Diseases_identifiers: https://diseases.jensenlab.org/Entity?documents=10&type1=9606&id1=ENSP00000369497&type2=-26&id2=DOID:15, MedlinePlus, MedlinePlus
   - Diseases_scores: 7.654, CURATED, CURATED
   - Sources: DISEASES
   - Diseases_sources: textmining, knowledge, knowledge
   - Diseases_confidences: 3.827, 5.0, 5.0

57. **pancreas disease**
   - Diseases_identifiers: https://diseases.jensenlab.org/Entity?documents=10&type1=9606&id1=ENSP00000369497&type2=-26&id2=DOID:26
   - Diseases_scores: 6.296
   - Sources: DISEASES
   - Diseases_sources: textmining
   - Diseases_confidences: 3.148

58. **disease of anatomical entity**
   - Diseases_identifiers: https://diseases.jensenlab.org/Entity?documents=10&type1=9606&id1=ENSP00000369497&type2=-26&id2=DOID:7, MedlinePlus, MedlinePlus, MedlinePlus, MedlinePlus, MedlinePlus, UniProtKB-KW
   - Diseases_scores: 7.512, CURATED, CURATED, CURATED, CURATED, CURATED, CURATED
   - Sources: DISEASES
   - Diseases_sources: textmining, knowledge, knowledge, knowledge, knowledge, knowledge, knowledge
   - Diseases_confidences: 3.756, 5.0, 5.0, 5.0, 5.0, 5.0, 4.0

59. **reproductive organ cancer**
   - Diseases_identifiers: https://diseases.jensenlab.org/Entity?documents=10&type1=9606&id1=ENSP00000369497&type2=-26&id2=DOID:193, MedlinePlus, MedlinePlus
   - Diseases_scores: 7.721, CURATED, CURATED
   - Sources: DISEASES
   - Diseases_sources: textmining, knowledge, knowledge
   - Diseases_confidences: 3.86, 5.0, 5.0

60. **sporadic breast cancer**
   - Diseases_identifiers: https://diseases.jensenlab.org/Entity?documents=10&type1=9606&id1=ENSP00000369497&type2=-26&id2=DOID:8029
   - Diseases_scores: 6.763
   - Sources: DISEASES
   - Diseases_sources: textmining
   - Diseases_confidences: 3.381

61. **Fanconi anemia**
   - Diseases_identifiers: https://diseases.jensenlab.org/Entity?documents=10&type1=9606&id1=ENSP00000369497&type2=-26&id2=DOID:13636, MedlinePlus, UniProtKB-KW
   - Diseases_scores: 7.109, CURATED, CURATED
   - Sources: DISEASES
   - Diseases_sources: textmining, knowledge, knowledge
   - Diseases_confidences: 3.555, 5.0, 4.0

62. **Lynch syndrome**
   - Diseases_identifiers: https://diseases.jensenlab.org/Entity?documents=10&type1=9606&id1=ENSP00000369497&type2=-26&id2=DOID:3883
   - Diseases_scores: 6.732
   - Sources: DISEASES
   - Diseases_sources: textmining
   - Diseases_confidences: 3.366

63. **adenocarcinoma**
   - Diseases_identifiers: https://diseases.jensenlab.org/Entity?documents=10&type1=9606&id1=ENSP00000369497&type2=-26&id2=DOID:299, MedlinePlus
   - Diseases_scores: 6.125, CURATED
   - Sources: DISEASES
   - Diseases_sources: textmining, knowledge
   - Diseases_confidences: 3.063, 5.0

64. **bile duct adenocarcinoma**
   - Diseases_identifiers: MedlinePlus
   - Diseases_scores: CURATED
   - Sources: DISEASES
   - Diseases_sources: knowledge
   - Diseases_confidences: 5.0

65. **autosomal dominant disease**
   - Diseases_identifiers: https://diseases.jensenlab.org/Entity?documents=10&type1=9606&id1=ENSP00000369497&type2=-26&id2=DOID:0050736
   - Diseases_scores: 6.694
   - Sources: DISEASES
   - Diseases_sources: textmining
   - Diseases_confidences: 3.347

66. **male reproductive system disease**
   - Diseases_identifiers: https://diseases.jensenlab.org/Entity?documents=10&type1=9606&id1=ENSP00000369497&type2=-26&id2=DOID:48, MedlinePlus
   - Diseases_scores: 6.516, CURATED
   - Sources: DISEASES
   - Diseases_sources: textmining, knowledge
   - Diseases_confidences: 3.258, 5.0

67. **female breast cancer**
   - Diseases_identifiers: https://diseases.jensenlab.org/Entity?documents=10&type1=9606&id1=ENSP00000369497&type2=-26&id2=DOID:0050671
   - Diseases_scores: 6.103
   - Sources: DISEASES
   - Diseases_sources: textmining
   - Diseases_confidences: 3.051

68. **bilateral breast cancer**
   - Diseases_identifiers: https://diseases.jensenlab.org/Entity?documents=10&type1=9606&id1=ENSP00000369497&type2=-26&id2=DOID:6741
   - Diseases_scores: 6.590
   - Sources: DISEASES
   - Diseases_sources: textmining
   - Diseases_confidences: 3.295

69. **bile duct carcinoma**
   - Diseases_identifiers: MedlinePlus
   - Diseases_scores: CURATED
   - Sources: DISEASES
   - Diseases_sources: knowledge
   - Diseases_confidences: 5.0

70. **cystadenocarcinoma**
   - Diseases_identifiers: https://diseases.jensenlab.org/Entity?documents=10&type1=9606&id1=ENSP00000369497&type2=-26&id2=DOID:3111
   - Diseases_scores: 6.111
   - Sources: DISEASES
   - Diseases_sources: textmining
   - Diseases_confidences: 3.056

71. **thoracic cancer**
   - Diseases_identifiers: https://diseases.jensenlab.org/Entity?documents=10&type1=9606&id1=ENSP00000369497&type2=-26&id2=DOID:5093, MedlinePlus
   - Diseases_scores: 7.916, CURATED
   - Sources: DISEASES
   - Diseases_sources: textmining, knowledge
   - Diseases_confidences: 3.958, 5.0

72. **female reproductive organ cancer**
   - Diseases_identifiers: https://diseases.jensenlab.org/Entity?documents=10&type1=9606&id1=ENSP00000369497&type2=-26&id2=DOID:120, MedlinePlus
   - Diseases_scores: 7.745, CURATED
   - Sources: DISEASES
   - Diseases_sources: textmining, knowledge
   - Diseases_confidences: 3.873, 5.0

73. **thoracic disease**
   - Diseases_identifiers: https://diseases.jensenlab.org/Entity?documents=10&type1=9606&id1=ENSP00000369497&type2=-26&id2=DOID:0060118, MedlinePlus
   - Diseases_scores: 7.912, CURATED
   - Sources: DISEASES
   - Diseases_sources: textmining, knowledge
   - Diseases_confidences: 3.956, 5.0

74. **malignant adenoma**
   - Diseases_identifiers: https://diseases.jensenlab.org/Entity?documents=10&type1=9606&id1=ENSP00000369497&type2=-26&id2=DOID:0080364
   - Diseases_scores: 6.104
   - Sources: DISEASES
   - Diseases_sources: textmining
   - Diseases_confidences: 3.052

75. **serous cystadenocarcinoma**
   - Diseases_identifiers: https://diseases.jensenlab.org/Entity?documents=10&type1=9606&id1=ENSP00000369497&type2=-26&id2=DOID:3114
   - Diseases_scores: 6.124
   - Sources: DISEASES
   - Diseases_sources: textmining
   - Diseases_confidences: 3.062

76. **genetic disease**
   - Diseases_identifiers: https://diseases.jensenlab.org/Entity?documents=10&type1=9606&id1=ENSP00000369497&type2=-26&id2=DOID:630
   - Diseases_scores: 6.433
   - Sources: DISEASES
   - Diseases_sources: textmining
   - Diseases_confidences: 3.217

77. **autosomal genetic disease**
   - Diseases_identifiers: https://diseases.jensenlab.org/Entity?documents=10&type1=9606&id1=ENSP00000369497&type2=-26&id2=DOID:0050739
   - Diseases_scores: 6.724
   - Sources: DISEASES
   - Diseases_sources: textmining
   - Diseases_confidences: 3.362

78. **Li-Fraumeni syndrome**
   - Diseases_identifiers: https://diseases.jensenlab.org/Entity?documents=10&type1=9606&id1=ENSP00000369497&type2=-26&id2=DOID:3012
   - Diseases_scores: 6.134
   - Sources: DISEASES
   - Diseases_sources: textmining
   - Diseases_confidences: 3.067

79. **disease**
   - Diseases_identifiers: https://diseases.jensenlab.org/Entity?documents=10&type1=9606&id1=ENSP00000369497&type2=-26&id2=DOID:4, MedlinePlus, MedlinePlus, MedlinePlus, MedlinePlus, MedlinePlus, UniProtKB-KW
   - Diseases_scores: 7.501, CURATED, CURATED, CURATED, CURATED, CURATED, CURATED
   - Sources: DISEASES
   - Diseases_sources: textmining, knowledge, knowledge, knowledge, knowledge, knowledge, knowledge
   - Diseases_confidences: 3.751, 5.0, 5.0, 5.0, 5.0, 5.0, 4.0

80. **male breast cancer**
   - Diseases_identifiers: https://diseases.jensenlab.org/Entity?documents=10&type1=9606&id1=ENSP00000369497&type2=-26&id2=DOID:1614
   - Diseases_scores: 7.010
   - Sources: DISEASES
   - Diseases_sources: textmining
   - Diseases_confidences: 3.505

81. **aplastic anemia**
   - Diseases_identifiers: https://diseases.jensenlab.org/Entity?documents=10&type1=9606&id1=ENSP00000369497&type2=-26&id2=DOID:12449, MedlinePlus, UniProtKB-KW
   - Diseases_scores: 6.906, CURATED, CURATED
   - Sources: DISEASES
   - Diseases_sources: textmining, knowledge, knowledge
   - Diseases_confidences: 3.453, 5.0, 4.0

82. **monogenic disease**
   - Diseases_identifiers: https://diseases.jensenlab.org/Entity?documents=10&type1=9606&id1=ENSP00000369497&type2=-26&id2=DOID:0050177
   - Diseases_scores: 6.672
   - Sources: DISEASES
   - Diseases_sources: textmining
   - Diseases_confidences: 3.336

83. **disease of cellular proliferation**
   - Diseases_identifiers: https://diseases.jensenlab.org/Entity?documents=10&type1=9606&id1=ENSP00000369497&type2=-26&id2=DOID:14566, MedlinePlus, MedlinePlus, MedlinePlus, MedlinePlus, UniProtKB-KW
   - Diseases_scores: 7.798, CURATED, CURATED, CURATED, CURATED, CURATED
   - Sources: DISEASES
   - Diseases_sources: textmining, knowledge, knowledge, knowledge, knowledge, knowledge
   - Diseases_confidences: 3.899, 5.0, 5.0, 5.0, 5.0, 4.0

84. **bile duct cancer**
   - Diseases_identifiers: MedlinePlus
   - Diseases_scores: CURATED
   - Sources: DISEASES
   - Diseases_sources: knowledge
   - Diseases_confidences: 5.0

85. **malignant cystadenoma**
   - Diseases_identifiers: https://diseases.jensenlab.org/Entity?documents=10&type1=9606&id1=ENSP00000369497&type2=-26&id2=DOID:60004
   - Diseases_scores: 6.111
   - Sources: DISEASES
   - Diseases_sources: textmining
   - Diseases_confidences: 3.056

86. **gonadal disease**
   - Diseases_identifiers: https://diseases.jensenlab.org/Entity?documents=10&type1=9606&id1=ENSP00000369497&type2=-26&id2=DOID:2277, MedlinePlus
   - Diseases_scores: 7.773, CURATED
   - Sources: DISEASES
   - Diseases_sources: textmining, knowledge
   - Diseases_confidences: 3.887, 5.0

87. **hereditary breast ovarian cancer syndrome**
   - Diseases_identifiers: https://diseases.jensenlab.org/Entity?documents=10&type1=9606&id1=ENSP00000369497&type2=-26&id2=DOID:5683
   - Diseases_scores: 7.843
   - Sources: DISEASES
   - Diseases_sources: textmining
   - Diseases_confidences: 3.921

88. **female reproductive system disease**
   - Diseases_identifiers: https://diseases.jensenlab.org/Entity?documents=10&type1=9606&id1=ENSP00000369497&type2=-26&id2=DOID:229, MedlinePlus
   - Diseases_scores: 7.673, CURATED
   - Sources: DISEASES
   - Diseases_sources: textmining, knowledge
   - Diseases_confidences: 3.837, 5.0

89. **endocrine gland cancer**
   - Diseases_identifiers: https://diseases.jensenlab.org/Entity?documents=10&type1=9606&id1=ENSP00000369497&type2=-26&id2=DOID:170
   - Diseases_scores: 6.286
   - Sources: DISEASES
   - Diseases_sources: textmining
   - Diseases_confidences: 3.143

90. **germ cell cancer**
   - Source: HPO

91. **gemistocytic astrocytoma**
   - Source: HPO

92. **hypertrophic cardiomyopathy**
   - Source: HPO

93. **astroblastoma**
   - Source: HPO

94. **astigmatism**
   - Source: HPO

95. **kidney disease**
   - Source: HPO

96. **microphthalmia**
   - Source: HPO

97. **gastrointestinal system disease**
   - Diseases_identifiers: https://diseases.jensenlab.org/Entity?documents=10&type1=9606&id1=ENSP00000369497&type2=-26&id2=DOID:77, MedlinePlus
   - Sources: DISEASES
   - Diseases_confidences: 3.09, 5.0
   - Diseases_scores: 6.180, CURATED
   - Source: HPO
   - Diseases_sources: textmining, knowledge

98. **hypertropia**
   - Source: HPO

99. **liver disease**
   - Source: HPO

100. **pathologic nystagmus**
   - Source: HPO

101. **leukopenia**
   - Source: HPO

102. **hypertension**
   - Source: HPO

103. **liver cancer**
   - Source: HPO

104. **arthropathy**
   - Source: HPO

105. **nervous system cancer**
   - Source: HPO

106. **kidney failure**
   - Source: HPO

107. **atrial heart septal defect**
   - Source: HPO

108. **aniridia**
   - Source: HPO

109. **visual impairment and progressive phthisis bulbi**
   - Source: HPO

110. **diabetes mellitus**
   - Source: HPO

111. **eye disease**
   - Source: HPO

112. **hematologic cancer**
   - Source: HPO

113. **paralytic ileus**
   - Source: HPO

114. **spontaneous ocular nystagmus**
   - Source: HPO

115. **Glanzmann's thrombasthenia**
   - Source: HPO

116. **ovarian cancer**
   - Diseases_identifiers: https://diseases.jensenlab.org/Entity?documents=10&type1=9606&id1=ENSP00000369497&type2=-26&id2=DOID:2394, MedlinePlus
   - Sources: DISEASES
   - Diseases_confidences: 3.928, 5.0
   - Diseases_scores: 7.857, CURATED
   - Source: HPO
   - Diseases_sources: textmining, knowledge

117. **acute myeloid leukemia without maturation**
   - Source: HPO

118. **hypogonadism**
   - Source: HPO

119. **patent ductus arteriosus**
   - Source: HPO

120. **endocrine system disease**
   - Diseases_identifiers: https://diseases.jensenlab.org/Entity?documents=10&type1=9606&id1=ENSP00000369497&type2=-26&id2=DOID:28, MedlinePlus
   - Sources: DISEASES
   - Diseases_confidences: 3.789, 5.0
   - Diseases_scores: 7.578, CURATED
   - Source: HPO
   - Diseases_sources: textmining, knowledge

121. **bone marrow disease**
   - Source: HPO

122. **melanoma**
   - Source: HPO

123. **pilocytic astrocytoma**
   - Source: HPO

124. **hepatobiliary disease**
   - Diseases_identifiers: MedlinePlus
   - Sources: DISEASES
   - Diseases_confidences: 5.0
   - Diseases_scores: CURATED
   - Source: HPO
   - Diseases_sources: knowledge

125. **intestinal disease**
   - Diseases_identifiers: https://diseases.jensenlab.org/Entity?documents=10&type1=9606&id1=ENSP00000369497&type2=-26&id2=DOID:5295
   - Sources: DISEASES
   - Diseases_confidences: 3.027
   - Diseases_scores: 6.053
   - Source: HPO
   - Diseases_sources: textmining

126. **hematopoietic system disease**
   - Diseases_identifiers: https://diseases.jensenlab.org/Entity?documents=10&type1=9606&id1=ENSP00000369497&type2=-26&id2=DOID:74, MedlinePlus, UniProtKB-KW
   - Sources: DISEASES
   - Diseases_confidences: 3.12, 5.0, 4.0
   - Diseases_scores: 6.241, CURATED, CURATED
   - Source: HPO
   - Diseases_sources: textmining, knowledge, knowledge

127. **cleft palate**
   - Source: HPO

128. **spinal cord disease**
   - Source: HPO

129. **colon adenocarcinoma**
   - Source: HPO

130. **choanal atresia**
   - Source: HPO

131. **anemia**
   - Diseases_identifiers: https://diseases.jensenlab.org/Entity?documents=10&type1=9606&id1=ENSP00000369497&type2=-26&id2=DOID:2355, MedlinePlus, UniProtKB-KW
   - Sources: DISEASES
   - Diseases_confidences: 3.18, 5.0, 4.0
   - Diseases_scores: 6.360, CURATED, CURATED
   - Source: HPO
   - Diseases_sources: textmining, knowledge, knowledge

132. **congenital heart disease**
   - Source: HPO

133. **juvenile pilocytic astrocytoma**
   - Source: HPO

134. **azoospermia**
   - Source: HPO

135. **central nervous system cancer**
   - Source: HPO

136. **subependymal giant cell astrocytoma**
   - Source: HPO

137. **acute myeloid leukemia**
   - Source: HPO

138. **heart valve disease**
   - Source: HPO

139. **colon cancer**
   - Source: HPO

140. **Meckel's diverticulum**
   - Source: HPO

141. **heart septal defect**
   - Source: HPO

142. **lung disease**
   - Source: HPO

143. **exocrine pancreatic insufficiency**
   - Source: HPO

144. **cryptorchidism**
   - Source: HPO

145. **colorectal cancer**
   - Diseases_identifiers: https://diseases.jensenlab.org/Entity?documents=10&type1=9606&id1=ENSP00000369497&type2=-26&id2=DOID:9256
   - Sources: DISEASES
   - Diseases_confidences: 3.015
   - Diseases_scores: 6.030
   - Source: HPO
   - Diseases_sources: textmining

146. **lymphatic system disease**
   - Source: HPO

147. **neuroectodermal tumor**
   - Source: HPO

148. **enterocele**
   - Source: HPO

149. **childhood cerebral astrocytoma**
   - Source: HPO

150. **cranial nerve palsy**
   - Source: HPO

151. **central nervous system disease**
   - Source: HPO

152. **colonic benign neoplasm**
   - Source: HPO

153. **protoplasmic astrocytoma**
   - Source: HPO

154. **anaplastic astrocytoma**
   - Source: HPO

155. **congenital vertical talus**
   - Source: HPO

156. **glioblastoma**
   - Source: HPO

157. **lung benign neoplasm**
   - Source: HPO

158. **biliary tract disease**
   - Diseases_identifiers: MedlinePlus
   - Sources: DISEASES
   - Diseases_confidences: 5.0
   - Diseases_scores: CURATED
   - Source: HPO
   - Diseases_sources: knowledge

159. **tetralogy of Fallot**
   - Source: HPO

160. **high grade glioma**
   - Source: HPO

161. **cardiomyopathy**
   - Source: HPO

162. **acute myeloid leukemia with maturation**
   - Source: HPO

163. **sideroblastic anemia**
   - Source: HPO

164. **leukemia**
   - Source: HPO

165. **mechanical strabismus**
   - Source: HPO

166. **childhood medulloblastoma**
   - Source: HPO

167. **cholestasis**
   - Source: HPO

168. **mixed glioma**
   - Source: HPO

169. **melanotic medulloblastoma**
   - Source: HPO

170. **dissociated nystagmus**
   - Source: HPO

171. **colorectal carcinoma**
   - Source: HPO

172. **urinary system benign neoplasm**
   - Source: HPO

173. **hydrocephalus**
   - Source: HPO

174. **adult medulloblastoma**
   - Source: HPO

175. **syndactyly**
   - Source: HPO

176. **malignant astrocytoma**
   - Source: HPO

177. **myelodysplastic syndrome**
   - Source: HPO

178. **obstructive hydrocephalus**
   - Source: HPO

179. **megacolon**
   - Source: HPO

180. **imperforate anus**
   - Source: HPO

181. **biliary tract cancer**
   - Diseases_identifiers: MedlinePlus
   - Sources: DISEASES
   - Diseases_confidences: 5.0
   - Diseases_scores: CURATED
   - Source: HPO
   - Diseases_sources: knowledge

182. **intestinal pseudo-obstruction**
   - Source: HPO

183. **ptosis**
   - Source: HPO

184. **Hirschsprung's disease**
   - Source: HPO

185. **oligohydramnios**
   - Source: HPO

186. **lens disease**
   - Source: HPO

187. **movement disease**
   - Source: HPO

188. **spina bifida**
   - Source: HPO

189. **clubfoot**
   - Source: HPO

190. **medullomyoblastoma**
   - Source: HPO

191. **duodenal atresia**
   - Source: HPO

192. **ovarian disease**
   - Diseases_identifiers: https://diseases.jensenlab.org/Entity?documents=10&type1=9606&id1=ENSP00000369497&type2=-26&id2=DOID:1100, MedlinePlus
   - Sources: DISEASES
   - Diseases_confidences: 3.905, 5.0
   - Diseases_scores: 7.810, CURATED
   - Source: HPO
   - Diseases_sources: textmining, knowledge

193. **fibrillary astrocytoma**
   - Source: HPO

194. **embryonal cancer**
   - Source: HPO

195. **communicating hydrocephalus**
   - Source: HPO

196. **germ cell and embryonal cancer**
   - Source: HPO

197. **cardiovascular system disease**
   - Source: HPO

198. **gliomatosis cerebri**
   - Source: HPO

199. **extrahepatic cholestasis**
   - Source: HPO

200. **kidney cancer**
   - Source: HPO

201. **tracheal disease**
   - Source: HPO

202. **thrombocytopenia**
   - Source: HPO

203. **giant cell glioblastoma**
   - Source: HPO

204. **cancer**
   - Diseases_identifiers: https://diseases.jensenlab.org/Entity?documents=10&type1=9606&id1=ENSP00000369497&type2=-26&id2=DOID:162, MedlinePlus, MedlinePlus, MedlinePlus, MedlinePlus, UniProtKB-KW
   - Sources: DISEASES
   - Diseases_confidences: 3.903, 5.0, 5.0, 5.0, 5.0, 4.0
   - Diseases_scores: 7.805, CURATED, CURATED, CURATED, CURATED, CURATED
   - Source: HPO
   - Diseases_sources: textmining, knowledge, knowledge, knowledge, knowledge, knowledge

205. **pancreatic cancer**
   - Diseases_identifiers: https://diseases.jensenlab.org/Entity?documents=10&type1=9606&id1=ENSP00000369497&type2=-26&id2=DOID:1793
   - Sources: DISEASES
   - Diseases_confidences: 3.2
   - Diseases_scores: 6.400
   - Source: HPO
   - Diseases_sources: textmining

206. **cranial nerve disease**
   - Source: HPO

207. **breast carcinoma**
   - Diseases_identifiers: https://diseases.jensenlab.org/Entity?documents=10&type1=9606&id1=ENSP00000369497&type2=-26&id2=DOID:3459
   - Sources: DISEASES, GWAS
   - Diseases_confidences: 3.32
   - Gwas_pvalue: 3e-15
   - Diseases_scores: 6.640
   - Diseases_sources: textmining

208. **breast cancer**
   - Diseases_identifiers: https://diseases.jensenlab.org/Entity?documents=10&type1=9606&id1=ENSP00000369497&type2=-26&id2=DOID:1612, MedlinePlus
   - Sources: OMIM, DISEASES
   - Diseases_confidences: 3.959, 5.0
   - O_inheritance: AD,SOMATIC
   - O_modifiers: SUSCEPTIBILITY
   - Diseases_scores: 7.917, CURATED
   - Diseases_sources: textmining, knowledge

209. **medulloblastoma**
   - Sources: OMIM
   - O_inheritance: AD,AR,SOMATIC
   - O_modifiers: 

210. **Fanconi anemia complementation group D1**
   - Sources: OMIM
   - O_inheritance: AR
   - O_modifiers: 

211. **prostate cancer**
   - Diseases_identifiers: https://diseases.jensenlab.org/Entity?documents=10&type1=9606&id1=ENSP00000369497&type2=-26&id2=DOID:10283, MedlinePlus
   - Sources: OMIM, DISEASES
   - Diseases_confidences: 3.259, 5.0
   - O_inheritance: AD,XL,SOMATIC
   - O_modifiers: 
   - Diseases_scores: 6.517, CURATED
   - Diseases_sources: textmining, knowledge

212. **Abnormality of the skin**
   - Source: HPO

213. **Muscle Hypotonia**
   - Source: HPO

214. **Headache**
   - Source: HPO

215. **Hyperkinesis**
   - Source: HPO

216. **Cognitive impairment**
   - Source: HPO

217. **Flexion contracture**
   - Source: HPO

218. **Hallucinations**
   - Source: HPO

219. **Basal cell carcinoma**
   - Source: HPO

220. **Memory Disorders**
   - Source: HPO

221. **Amaurosis Fugax**
   - Source: HPO

222. **Constipation**
   - Source: HPO

223. **Dysarthria**
   - Source: HPO

224. **Muscle Hypertonia**
   - Source: HPO

225. **Paresthesia**
   - Source: HPO

226. **Abnormality of the anterior pituitary**
   - Source: HPO

227. **Anorectal anomaly**
   - Source: HPO

228. **Hypesthesia**
   - Source: HPO

229. **Gastrointestinal hemorrhage**
   - Source: HPO

230. **Anxiety**
   - Source: HPO

231. **Behavioral abnormality**
   - Source: HPO

232. **Behavioral abnormality**
   - Source: HPO

233. **Abnormality of the nervous system**
   - Source: HPO

234. **Abnormal foot morphology**
   - Source: HPO

235. **Abdominal Pain**
   - Source: HPO

236. **Intrauterine growth retardation**
   - Source: HPO

237. **Scoliosis**
   - Source: HPO

238. **Aplasia/Hypoplasia of the mandible**
   - Source: HPO

239. **Deafness**
   - Source: HPO

240. **Hypoplasia of the ulna**
   - Source: HPO

241. **Micrognathia**
   - Source: HPO

242. **T-cell acute lymphoblastic leukemias**
   - Source: HPO

243. **Anorexia**
   - Source: HPO

244. **Abnormality of the cardiovascular system**
   - Source: HPO

245. **Splenomegaly**
   - Source: HPO

246. **Vision Disorders**
   - Source: HPO

247. **Neoplasm of the nervous system**
   - Source: HPO

248. **Lymphadenopathy**
   - Source: HPO

249. **Proptosis**
   - Source: HPO

250. **Neoplasm of the respiratory system**
   - Source: HPO

251. **Abnormal heart valve morphology**
   - Source: HPO

252. **Neoplasm of the genitourinary tract**
   - Source: HPO

253. **Intellectual Disability**
   - Source: HPO

254. **Abnormality of the eye**
   - Source: HPO

255. **Hypertelorism**
   - Source: HPO

256. **Weight Loss**
   - Source: HPO

257. **Fatigue**
   - Source: HPO

258. **Reflex, Abnormal**
   - Source: HPO

259. **Abnormality of the skin**
   - Source: HPO

260. **Diarrhea**
   - Source: HPO

261. **Back Pain**
   - Source: HPO

262. **Arteriovenous malformation**
   - Source: HPO

263. **Abnormality of the hand**
   - Source: HPO

264. **Abnormality of the pancreas**
   - Source: HPO

265. **Hypopigmentation of the skin**
   - Source: HPO

266. **Body Temperature Changes**
   - Source: HPO

267. **Neural tube defect**
   - Source: HPO

268. **Multiple cafe-au-lait spots**
   - Source: HPO

269. **Intrauterine growth retardation**
   - Source: HPO

270. **Abnormal thumb morphology**
   - Source: HPO

271. **Abnormality of chromosome stability**
   - Source: HPO

272. **Tracheoesophageal fistula**
   - Source: HPO

273. **Hip dislocation**
   - Source: HPO

274. **Cafe-au-Lait Spots**
   - Source: HPO

275. **Abnormality of the uterus**
   - Source: HPO

276. **Triphalangeal thumb**
   - Source: HPO

277. **Thinness**
   - Source: HPO

278. **Pain**
   - Source: HPO

279. **Hearing Loss**
   - Source: HPO

280. **Cataract**
   - Source: HPO

281. **Abnormality of the mouth**
   - Source: HPO

282. **Vision, Low**
   - Source: HPO

283. **Abnormality of the genitourinary system**
   - Source: HPO

284. **Hyperpigmentation of the skin**
   - Source: HPO

285. **Jaundice**
   - Source: HPO

286. **Hepatomegaly**
   - Source: HPO

287. **Fever**
   - Source: HPO

288. **Hematuria**
   - Source: HPO

289. **Pruritus**
   - Source: HPO

290. **Facial asymmetry**
   - Source: HPO

291. **Abnormality of the gastrointestinal tract**
   - Source: HPO

292. **Joint dislocation**
   - Source: HPO

